# Factory Planning Example

## Objective and Prerequisites

This program is based on the example found on github here: https://github.com/Gurobi/modeling-examples/blob/master/hp_williams/factory_planning_2.ipynb

This model is an example of a production planning problem

### Problem Description

#### Products

- The Factory makes 3 products: Prod1, Prod2 and Prod3

#### Machines

It uses the following machines:
    - Grinders - Quantity = 4
    - Drills - Quantity = 2
    - Borer - Quantity = 1

#### Profit Contribution and Manufacturing Time

- Each product has a defined profit contribution per unit sold = Sales Price minus Cost of Raw Mat.
- Each product requires a certain amount of time on each machine in hours

|  | Prod1 | Prod2 | Prod3 |
| --- | --- | --- | --- |
| PROFIT | 10 | 6 | 8 |
| Grinders | 0.5 | 0.7 | - |
| Drills | 0.1 | 0.2 | 0.3 |
| Borer | 0.05 | - | 0.03 |

#### Maintenance Schedule

- Each machine must be down for maintenance in 1 month out of 6 months

#### Sale Limitation/Demand Forecast

Limitation of the number of products that can be sold in a given month - this could be descibed as the forecast planning shown below

| Month | Prod1 | Prod2 | Prod3 |
| --- | --- | --- | --- |
| Jan | 500 | 1000 | 300 |
| Feb | 600 | 500 | 200 |
| Mar | 300 | 600 | 0 |
| Apr | 200 | 300 | 400 |
| May | 0 | 100 | 500 |
| Jun | 500 | 500 | 100 |

#### Inventory

- Upto 100 units of each product may be stored in inventory at a cost of $0.50 per unit per month.
- At the start of January, there is no product inventory
- By end of June, there should be 50 units of each product in Inventory

#### Factory Schedule

- Production is scheduled at 6 days a week with 3 eight hour shifts per day.
- Each month has 24 working days

### Output Required

- What is the maximum profit that can be achieved?
- What should the Production plan be?
- What should the tool maintenance schedule be?

## Modelling

### Sets and Indices

t ∊ Months = {Jan, Feb, Mar, Apr, May, Jun}

p ∊ Products = {Prod1, Prod2, Prod3}

m ∊ Machines = {Grinders, Drills, Borer}

### Parameters

hours_per_month -> Tme (in hours/month) available at any machine on a monthly basis = Number of working days in a month (24) * number of shifts (3) * duration of a shift (8)

max_inventory -> Maximum number of units of a single product type that can be stored in inventory at any given month

holding_cost -> monthly cost in (USD/unit/month) of keeping in inventory a unit of any product type

store_target -> Number of units of each product type to keep in inventory at the end of a planning phase

profit(p) -> Proft (in USD/unit) of product p

installed(m) -> Number of machines of type m installed in the factory

down_req(m) -> Number of machines of type m that should be shceduled for maintenance at some point in the planning phase

time_req (m,p) -> Time (in hours/unit) needed on machine m to manufacture one unit of product p

max_sales(t,p) -> Number of units of product p that can be sold in month t

### Decision Variables

make(t,p) -> Number of units of product p to manufacture in month t

store(t,p)[0, max_inventory] -> Number of units of product p to store in month t

sell(t,p)[0, max_sales(t,p)] -> Number of units of product p to sell in month t

repair(t,m) {0,1,...., down_req(m)} -> Number of machines of type m scheduled for maintenance in month t

### Assumption - We can produce fractional units

### Objective Function - Profit. Maximize total profit during planning phase

### Constraints

- __Initial Balance:__ For each product p, the number of units produced should be equal to the number of units sold plus the number of units stored. Here since Jan is used since it is the first month

        make(Jan,p) = sell(t,p) + store(Jan,p)

- __Balance:__ For each product p, the number of units produced in a month t and previously stored should be equal to the number of units sold and stored in that month
    
        store(t-1, p) + make(t,p) = sell(t,p) + store(t,p)

- __Machine Capacity:__ Total time to used to manufacture any product at machine type m cannot exceed its monthly capacity in hours

    ∑p time_req(m,p) * make(t,p) <= 
        hours_per_month * (installed(m) - repair(t,m))

- __Maintenance:__ The number of machines of type m schedule for maintenance should meet the requirement
    
    ∑t repair(t,m) = down_req(m)

### Implementation

In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

In [2]:
# Sets
products = ['Prod1', 'Prod2', 'Prod3']
machines = ['Grinders', 'Drills', 'Borers']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']

In [3]:
profit = {'Prod1':10, 'Prod2':6, 'Prod3':8}

In [4]:
time_req = {
    'Grinders': {'Prod1': 0.5, 'Prod2': 0.7},
    'Drills': {'Prod1': 0.1, 'Prod2': 0.2, 'Prod3': 0.3},
    'Borers': {'Prod1': 0.05, 'Prod3': 0.03}
}

In [5]:
# Number of each machine available
installed = {'Grinders': 4, 'Drills': 2, 'Borers': 1}

In [6]:
# All machines need to be down at least one month 
# during this 6 month phase
down_req = {'Grinders': 4, 'Drills': 2, 'Borers': 1}

In [7]:
# Sales Forecast/Requirement
max_sales = {
    ('Jan', 'Prod1'): 500,
    ('Jan', 'Prod2'): 1000,
    ('Jan', 'Prod3'): 300,
    ('Feb', 'Prod1'): 600,
    ('Feb', 'Prod2'): 500,
    ('Feb', 'Prod3'): 200,
    ('Mar', 'Prod1'): 300,
    ('Mar', 'Prod2'): 600,
    ('Mar', 'Prod3'): 0,
    ('Apr', 'Prod1'): 200,
    ('Apr', 'Prod2'): 300,
    ('Apr', 'Prod3'): 400,
    ('May', 'Prod1'): 0,
    ('May', 'Prod2'): 100,
    ('May', 'Prod3'): 500,
    ('Jun', 'Prod1'): 500,
    ('Jun', 'Prod2'): 500,
    ('Jun', 'Prod3'): 100,
}

In [8]:
holding_cost = 0.5
max_inventory = 100
store_target = 50
hours_per_month = 3*8*24

### Model Deployment

- For each product and each time period, variables are created for the amount of which product will be __Manufactured, Held and Sold__
- In each month, there is an upper limit on the number of units of each product can be sold.
- For each type of machine and each month, variable d defines how many machines are down during the month of this type

In [9]:
# Create the Model
factory = gp.Model('Factory Planning')

# Add the Variables

# Qty manufactured
make = factory.addVars(months, products, name='Make')

# Quantity stored
store = factory.addVars(months, products, ub=max_inventory, name='Store')

# Quantity Sold
sell = factory.addVars(months, products, ub=max_sales, name='Sell')

# Number of Machines Down
repair = factory.addVars(months, machines, vtype=GRB.INTEGER, 
                        ub=down_req, name='Repair')


Using license file /Users/mayukh/gurobi.lic
Academic license - for non-commercial use only


In [10]:
# Add the Constraints

# Initial Balance
initial_balance = factory.addConstrs((
    make[months[0], product] == 
    sell[months[0], product] + store[months[0], product] 
    for product in products), name='Initial_Balance')

In [12]:
# Balance
balance = factory.addConstrs((
    store[months[months.index(month)-1], product] + make[month, product] ==
    sell[month, product] + store[month, product]
    for product in products for month in months
    if month != months[0]), name='Balance')

The following constraint enforces that at the end of the last month the storage contains the specified amount of each product

In [14]:
target_inventory = factory.addConstrs((
    store[months[-1], product] == store_target
    for product in products), name='End_Balance')

In [16]:
# Machine Capacity
machine_cap = factory.addConstrs((
    gp.quicksum(time_req[machine][product] * make[month, product]
               for product in time_req[machine])
    <= hours_per_month * (installed[machine] - repair[month, machine])
    for machine in machines for month in months
    ), name='Capacity')

In [17]:
# maintenance
# Add the repair variable as part of the optimization
maint = factory.addConstrs((repair.sum('*', machine) == 
                           down_req[machine] for machine in machines),
                          'Maintenance')

In [19]:
# Define the objective
obj = gp.quicksum(profit[product] * sell[month, product] - 
                 holding_cost * store[month, product]
                 for month in months for product in products)

factory.setObjective(obj, GRB.MAXIMIZE)

In [20]:
factory.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 42 rows, 72 columns and 150 nonzeros
Model fingerprint: 0x5d70719c
Variable types: 54 continuous, 18 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-02, 6e+02]
  Objective range  [5e-01, 1e+01]
  Bounds range     [1e+02, 1e+03]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective -75.0000000
Presolve removed 6 rows and 8 columns
Presolve time: 0.00s
Presolved: 36 rows, 64 columns, 140 nonzeros
Variable types: 46 continuous, 18 integer (6 binary)

Root relaxation: objective 5.092500e+04, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 50925.0000    0    6  -75.00000 50925.0000      -     -    0s
H    0     0                    43525.000000 50925.0000  17.0%     -    0s
H    0     0                    45925.000000 50925.0000  10.9%     

### Production Plan - The number of each product to be made per month

In [24]:
rows = months.copy()
columns = products.copy()
make_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

In [27]:
for month, product in make.keys():
    if(abs(make[month, product].x) > 1e-6):
        make_plan.loc[month, product] = np.round(make[month, product].x, 1)

In [28]:
make_plan

,Prod1,Prod2,Prod3
Jan,500.0,1000.0,300.0
Feb,700.0,500.0,200.0
Mar,0.0,600.0,0.0
Apr,200.0,300.0,400.0
May,0.0,100.0,500.0
Jun,550.0,550.0,150.0


### Sales Plan

In [29]:
rows = months.copy()
columns = products.copy()
sell_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

In [30]:
for month, product in sell.keys():
    if (abs(sell[month, product].x) > 1e-6):
        sell_plan.loc[month, product] = np.round(sell[month, product].x, 1)
sell_plan

,Prod1,Prod2,Prod3
Jan,500.0,1000.0,300.0
Feb,600.0,500.0,200.0
Mar,100.0,600.0,0.0
Apr,200.0,300.0,400.0
May,0.0,100.0,500.0
Jun,500.0,500.0,100.0


### Inventory Plan

In [31]:
rows = months.copy()
columns = products.copy()
store_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in store.keys():
    if (abs(store[month, product].x) > 1e-6):
        store_plan.loc[month, product] = np.round(store[month, product].x, 1)
store_plan

,Prod1,Prod2,Prod3
Jan,0.0,0.0,0.0
Feb,100.0,0.0,0.0
Mar,0.0,0.0,0.0
Apr,0.0,0.0,0.0
May,0.0,0.0,0.0
Jun,50.0,50.0,50.0


### Maintenance Plan

In [32]:
rows = months.copy()
columns = machines.copy()
repair_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, machine in repair.keys():
    if (abs(repair[month, machine].x) > 1e-6):
        repair_plan.loc[month, machine] = repair[month, machine].x
repair_plan

,Grinders,Drills,Borers
Jan,0.0,0.0,0.0
Feb,0.0,0.0,0.0
Mar,0.0,1.0,1.0
Apr,0.0,0.0,0.0
May,2.0,0.0,0.0
Jun,2.0,1.0,0.0


In [33]:
factory.write('factoryplanning.sol')